# Imports

In [1]:
import pandas as pd
import numpy as np
from pycaret.classification import *
from sklearn.model_selection import train_test_split

# The dataset

In [ ]:
df = pd.read_csv("combined_simulated.csv")

In [93]:
# Convert 'UTC_TIME' column to datetime format and sort by time
df['UTC_TIME'] = pd.to_datetime(df['UTC_TIME'])
df.sort_values(by=['FLIGHT_INSTANCE', 'UTC_TIME'], inplace=True)

In [94]:
# Dropping irrelvant columns
df = df.drop(columns=['FLIGHT_PHASE_COUNT', 'Flight','MSN', 'FLIGHT_INSTANCE'])

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376554 entries, 0 to 337922
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   UTC_TIME                   376554 non-null  datetime64[ns]
 1   FUEL_USED_2                376554 non-null  float64       
 2   FUEL_USED_3                376554 non-null  float64       
 3   FUEL_USED_4                376554 non-null  float64       
 4   FW_GEO_ALTITUDE            376554 non-null  float64       
 5   VALUE_FOB                  376554 non-null  float64       
 6   VALUE_FUEL_QTY_CT          376554 non-null  float64       
 7   VALUE_FUEL_QTY_FT1         376554 non-null  float64       
 8   VALUE_FUEL_QTY_FT2         376554 non-null  float64       
 9   VALUE_FUEL_QTY_FT3         376554 non-null  float64       
 10  VALUE_FUEL_QTY_FT4         376554 non-null  float64       
 11  VALUE_FUEL_QTY_LXT         376554 non-null  float64      

# Modeling

In [140]:
# 2️⃣ Select all features (except the target)
features = df.columns.tolist()
features.remove('LEAK_FLOW_FLAG')  # Remove target column from features

# Target variable
target = 'LEAK_FLOW_FLAG'

In [141]:
# Create lag features (memory for time-series)
def create_lag_features(df, features, lags=[1, 3, 5, 10]):
    """
    Adds past values (lags) as new features to help capture time dependency.
    """
    df = df.copy()
    for lag in lags:
        for feature in features:
            df[f'{feature}_lag{lag}'] = df.groupby('FLIGHT_ID')[feature].shift(lag)
    return df

df = create_lag_features(df, features, lags=[1, 3, 5, 10]) 

In [142]:
# Drop rows with NaN values (due to lagging)
df.dropna(inplace=True)

In [143]:
# ROW LEVEL UNDER SAMPLING
# # Keeping 3x as many "No Leak" samples as leak samples
# from sklearn.utils import resample
# # Undersample "No Leak" Cases to Balance the Dataset
# df_leak = df[df["LEAK_FLOW_FLAG"] == 1]
# df_no_leak = df[df["LEAK_FLOW_FLAG"] == 0]

# # Keep 3x as many "No Leak" samples as "Leak" samples
# df_no_leak_balanced = resample(df_no_leak, 
#                                replace=False,  # No replacement (just reducing data)
#                                n_samples=len(df_leak) * 3,  
#                                random_state=42)

# Combine balanced dataset
#df_balanced = pd.concat([df_leak, df_no_leak_balanced])

In [210]:
# Count leaks per flight
flight_leak_counts = df.groupby("FLIGHT_ID")["LEAK_FLOW_FLAG"].sum()

# ✅ Compute the IQR to find flights with outlier-level leaks
Q1 = flight_leak_counts[flight_leak_counts > 0].quantile(0.25)
Q3 = flight_leak_counts[flight_leak_counts > 0].quantile(0.75)
IQR = Q3 - Q1

# Define outlier threshold (Q3 + 1.5 * IQR)
outlier_threshold = Q3 + (1.5 * IQR)
print(f"Outlier Threshold for Leaks per Flight: {outlier_threshold:.2f}")

# ✅ Select only flights where leaks are higher than the outlier threshold
flights_with_leaks = flight_leak_counts[flight_leak_counts > outlier_threshold].index

# ✅ Reduce No-Leak Flights Even More (1.5x the number of leak flights)
num_leak_flights = len(flights_with_leaks)  # Total number of high-leak flights
num_no_leak_flights = int(num_leak_flights * 1.5)  # Reduce no-leak flights even more

# ✅ Sample only the reduced number of no-leak flights
flights_no_leaks = flight_leak_counts[flight_leak_counts == 0].sample(
    n=num_no_leak_flights, 
    random_state=42
).index

# Keep only selected flights (Preserves sequences!)
df_balanced = df[df["FLIGHT_ID"].isin(flights_with_leaks.union(flights_no_leaks))]

Outlier Threshold for Leaks per Flight: 167.75


In [211]:
# Keep only selected flights (Preserves sequences!)
df_balanced["LEAK_FLOW_FLAG"].value_counts(normalize=True) * 100

LEAK_FLOW_FLAG
0    90.215155
1     9.784845
Name: proportion, dtype: float64

In [212]:
# 4️⃣ Perform Flight-Level Undersampling to Balance the Dataset

# Count leaks per flight
flight_leak_counts = df.groupby("FLIGHT_ID")["LEAK_FLOW_FLAG"].sum()

# ✅ Compute the IQR to find flights with outlier-level leaks
Q1 = flight_leak_counts[flight_leak_counts > 0].quantile(0.25)
Q3 = flight_leak_counts[flight_leak_counts > 0].quantile(0.75)
IQR = Q3 - Q1

# Define outlier threshold (Q3 + 1.5 * IQR)
outlier_threshold = Q3 + (1.5 * IQR)
print(f"Outlier Threshold for Leaks per Flight: {outlier_threshold:.2f}")

# ✅ Select only flights where leaks are higher than the outlier threshold
flights_with_leaks = flight_leak_counts[flight_leak_counts > outlier_threshold].index

# ✅ Further Reduce No-Leak Flights (Now 1.2x instead of 1.5x)
num_leak_flights = len(flights_with_leaks)  # Total number of high-leak flights
num_no_leak_flights = int(num_leak_flights * 1.2)  # Reduce no-leak flights even more

# ✅ Sample only the reduced number of no-leak flights
flights_no_leaks = flight_leak_counts[flight_leak_counts == 0].sample(
    n=num_no_leak_flights, 
    random_state=42
).index

# Keep only selected flights (Preserves sequences!)
df_balanced = df[df["FLIGHT_ID"].isin(flights_with_leaks.union(flights_no_leaks))]

Outlier Threshold for Leaks per Flight: 167.75


In [213]:
# Keep only selected flights (Preserves sequences!)
df_balanced["LEAK_FLOW_FLAG"].value_counts(normalize=True) * 100

LEAK_FLOW_FLAG
0    89.997596
1    10.002404
Name: proportion, dtype: float64

In [214]:
# 4️⃣ Perform Flight-Level Undersampling to Balance the Dataset

# Count leaks per flight
flight_leak_counts = df.groupby("FLIGHT_ID")["LEAK_FLOW_FLAG"].sum()

# ✅ Compute the IQR to find flights with outlier-level leaks
Q1 = flight_leak_counts[flight_leak_counts > 0].quantile(0.25)
Q3 = flight_leak_counts[flight_leak_counts > 0].quantile(0.75)
IQR = Q3 - Q1

# Define outlier threshold (Q3 + 1.5 * IQR)
outlier_threshold = Q3 + (1.5 * IQR)
print(f"Outlier Threshold for Leaks per Flight: {outlier_threshold:.2f}")

# ✅ Select only flights where leaks are higher than the outlier threshold
flights_with_leaks = flight_leak_counts[flight_leak_counts > outlier_threshold].index

# ✅ Aggressively Reduce No-Leak Flights (Now 1.0x instead of 1.2x)
num_leak_flights = len(flights_with_leaks)  # Total number of high-leak flights
num_no_leak_flights = int(num_leak_flights * 1.0)  # Keep only as many no-leak flights as leak flights

# ✅ Sample only the reduced number of no-leak flights
flights_no_leaks = flight_leak_counts[flight_leak_counts == 0].sample(
    n=num_no_leak_flights, 
    random_state=42
).index

# Keep only selected flights (Preserves sequences!)
df_balanced = df[df["FLIGHT_ID"].isin(flights_with_leaks.union(flights_no_leaks))]

Outlier Threshold for Leaks per Flight: 167.75


In [215]:
# Keep only selected flights (Preserves sequences!)
df_balanced["LEAK_FLOW_FLAG"].value_counts(normalize=True) * 100

LEAK_FLOW_FLAG
0    89.931648
1    10.068352
Name: proportion, dtype: float64

In [216]:
# Count leaks per flight
flight_leak_counts = df.groupby("FLIGHT_ID")["LEAK_FLOW_FLAG"].sum()

# ✅ Compute the IQR to find flights with outlier-level leaks
Q1 = flight_leak_counts[flight_leak_counts > 0].quantile(0.25)
Q3 = flight_leak_counts[flight_leak_counts > 0].quantile(0.75)
IQR = Q3 - Q1

# Define outlier threshold (Q3 + 1.5 * IQR)
outlier_threshold = Q3 + (1.5 * IQR)
print(f"Outlier Threshold for Leaks per Flight: {outlier_threshold:.2f}")

# ✅ Select only flights where leaks are higher than the outlier threshold
flights_with_leaks = flight_leak_counts[flight_leak_counts > outlier_threshold].index

# ✅ EXTREME Reduction: Only keep 0.5x as many no-leak flights as leak flights
num_leak_flights = len(flights_with_leaks)  # Total number of high-leak flights
num_no_leak_flights = int(num_leak_flights * 0.01)  # Cut down no-leak flights aggressively

# ✅ Sample only the reduced number of no-leak flights
flights_no_leaks = flight_leak_counts[flight_leak_counts == 0].sample(
    n=num_no_leak_flights, 
    random_state=42
).index

# Keep only selected flights (Preserves sequences!)
df_balanced = df[df["FLIGHT_ID"].isin(flights_with_leaks.union(flights_no_leaks))]

Outlier Threshold for Leaks per Flight: 167.75


In [217]:
# Keep only selected flights (Preserves sequences!)
df_balanced["LEAK_FLOW_FLAG"].value_counts(normalize=True) * 100

LEAK_FLOW_FLAG
0    86.996602
1    13.003398
Name: proportion, dtype: float64

In [218]:
# Count leaks per flight
flight_leak_counts = df.groupby("FLIGHT_ID")["LEAK_FLOW_FLAG"].sum()

# ✅ Compute the Mean Number of Leaks per Flight
mean_leaks_per_flight = flight_leak_counts[flight_leak_counts > 0].median()
print(f"Mean Number of Leaks per Flight: {mean_leaks_per_flight:.2f}")

# ✅ Select flights where the number of leaks is higher than the mean
flights_with_leaks = flight_leak_counts[flight_leak_counts > mean_leaks_per_flight].index

# ✅ EXTREME Reduction: Only keep 0.5x as many no-leak flights as leak flights
num_leak_flights = len(flights_with_leaks)  # Total number of high-leak flights
num_no_leak_flights = int(num_leak_flights * 0.01)  # Cut down no-leak flights aggressively

# ✅ Sample only the reduced number of no-leak flights
flights_no_leaks = flight_leak_counts[flight_leak_counts == 0].sample(
    n=num_no_leak_flights, 
    random_state=42
).index

# Keep only selected flights (Preserves sequences!)
df_balanced = df[df["FLIGHT_ID"].isin(flights_with_leaks.union(flights_no_leaks))]

Mean Number of Leaks per Flight: 33.00


In [219]:
# Keep only selected flights (Preserves sequences!)
df_balanced["LEAK_FLOW_FLAG"].value_counts(normalize=True) * 100

LEAK_FLOW_FLAG
0    88.509981
1    11.490019
Name: proportion, dtype: float64

In [226]:

# Count leaks per flight
flight_leak_counts = df.groupby("FLIGHT_ID")["LEAK_FLOW_FLAG"].sum()

# ✅ Compute the Mean Number of Leaks per Flight
mean_leaks_per_flight = flight_leak_counts[flight_leak_counts > 0].mean()
print(f"Mean Number of Leaks per Flight: {mean_leaks_per_flight:.2f}")

# ✅ Select flights where the number of leaks is higher than the median
flights_with_leaks = flight_leak_counts[flight_leak_counts > mean_leaks_per_flight].index

# ✅ EXTREME Reduction: Only keep 1% as many no-leak flights as leak flights
num_leak_flights = len(flights_with_leaks)  # Total number of flights with leaks
num_no_leak_flights = int(num_leak_flights * 0.01)  # Cut down no-leak flights aggressively

# ✅ Sample only the reduced number of no-leak flights
flights_no_leaks = flight_leak_counts[flight_leak_counts == 0].sample(
    n=num_no_leak_flights, 
    random_state=42
).index

# Keep only selected flights (Preserves sequences!)
df_balanced = df[df["FLIGHT_ID"].isin(flights_with_leaks.union(flights_no_leaks))]

Mean Number of Leaks per Flight: 66.87


In [227]:
# Keep only selected flights (Preserves sequences!)
df_balanced["LEAK_FLOW_FLAG"].value_counts(normalize=True) * 100

LEAK_FLOW_FLAG
0    85.797351
1    14.202649
Name: proportion, dtype: float64

In [228]:
df_balanced.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 35944 entries, 19979 to 345135
Data columns (total 631 columns):
 #    Column                                 Non-Null Count  Dtype         
---   ------                                 --------------  -----         
 0    UTC_TIME                               35944 non-null  datetime64[ns]
 1    FUEL_USED_2                            35944 non-null  float64       
 2    FUEL_USED_3                            35944 non-null  float64       
 3    FUEL_USED_4                            35944 non-null  float64       
 4    FW_GEO_ALTITUDE                        35944 non-null  float64       
 5    VALUE_FOB                              35944 non-null  float64       
 6    VALUE_FUEL_QTY_CT                      35944 non-null  float64       
 7    VALUE_FUEL_QTY_FT1                     35944 non-null  float64       
 8    VALUE_FUEL_QTY_FT2                     35944 non-null  float64       
 9    VALUE_FUEL_QTY_FT3                     35944 non

In [229]:
# Split data per FLIGHT_ID (to prevent leakage)
train_flights, test_flights = train_test_split(df_balanced['FLIGHT_ID'].unique(), test_size=0.2, random_state=42)

train_df = df_balanced[df_balanced['FLIGHT_ID'].isin(train_flights)]
test_df = df_balanced[df_balanced['FLIGHT_ID'].isin(test_flights)]

In [230]:
# Reset index (PyCaret needs a clean index)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [231]:
# # Initialize PyCaret classification module
clf_setup = setup(
    data=train_df, 
    target=target,
    train_size=0.8,
    session_id=42, 
    fold_strategy="stratifiedkfold", 
    fold=5
)

,Description,Value
0,Session id,42
1,Target,LEAK_FLOW_FLAG
2,Target type,Binary
3,Original data shape,"(26106, 631)"
4,Transformed data shape,"(26106, 1114)"
5,Transformed train set shape,"(20884, 1114)"
6,Transformed test set shape,"(5222, 1114)"
7,Numeric features,567
8,Date features,21
9,Categorical features,41


In [232]:
best_model = compare_models(sort='Recall')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.2061,0.6726,0.9385,0.1442,0.2499,0.0074,0.0308,3.5180
nb,Naive Bayes,0.7144,0.0000,0.5097,0.2496,0.3347,0.1795,0.1982,2.4340
dt,Decision Tree Classifier,0.8110,0.0000,0.3537,0.3375,0.3454,0.2350,0.2351,3.4940
svm,SVM - Linear Kernel,0.8132,0.7448,0.2825,0.2822,0.2405,0.1616,0.1762,3.3720
knn,K Neighbors Classifier,0.8303,0.0000,0.2035,0.3330,0.2526,0.1632,0.1695,3.1980
et,Extra Trees Classifier,0.8375,0.0000,0.1563,0.3352,0.2131,0.1357,0.1481,2.3480
lda,Linear Discriminant Analysis,0.8420,0.8054,0.1203,0.3363,0.1767,0.1103,0.1287,3.8640
rf,Random Forest Classifier,0.8475,0.0000,0.0907,0.3444,0.1435,0.0901,0.1148,4.0940
lightgbm,Light Gradient Boosting Machine,0.8486,0.0000,0.0683,0.3231,0.1126,0.0669,0.0917,3.3140
lr,Logistic Regression,0.8538,0.7703,0.0421,0.3457,0.0751,0.0459,0.0778,9.5000


In [233]:
# Tune the best model for better recall
# tuned_model = tune_model(best_model, optimize='Recall')